In [ ]:
import pandas as pd
import re
import numpy as np
from config import config
import requests
import csv
import WRIC_preprocessing as wric
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)


### Example Usage

In [ ]:
R1_metadata, R2_metadata, df_room1, df_room2 = wric.preprocess_WRIC_file("C:\Documents\WRIC_example_data\Results_1m_copy_anonymised.txt", code="id+comment", path_to_save=None) #path_to_save="C:\Documents\WRIC_example_data"
display(df_room1)

# do not include discrepancy check standard and explain that without calibration gases no realistic values (only do for actual person values)
#check_discrepancies(df_room2, threshold=0.05)

In [ ]:
preprocess_WRIC_files('id.csv', 'upload')

In [ ]:
import WRIC_preprocessing as wric

# Playground for Note Integration

In [ ]:
R1_metadata, R2_metadata, df_room1, df_room2 = wric.preprocess_WRIC_file("C:/Documents/WRIC_example_data/Results_1m_copy_anonymised.txt") 


In [ ]:
# read the note file into a pandas Dataframe
notes_path = "C:/Documents/WRIC_example_data/Main_note_yyyymmddxxxx.txt"
notes_content = wric.open_file(notes_path)

lines = [line.strip().split('\t') for line in notes_content[2:]]

df_note = pd.DataFrame(lines[2:], columns=lines[0])
df_note = df_note.dropna()

# combine to datetime
df_note['datetime'] = pd.to_datetime(df_note['Date'] + ' ' + df_note['Time'], format='%m/%d/%y %H:%M:%S')
df_note = df_note.drop(columns=['Date', 'Time'])

In [ ]:
display(df_note.head())

In [ ]:
start_sleeping = ["seng", "sleeping", "bed", "sove"]
stop_sleeping = ["vaagen", "vaekket", "awake", "wake"]
start_ree = ["start REE", "Start REE"] # or somehow both need to be present start, Start etc and REE, ree...
stop_ree = ["stop REE"]
start_eating = ["start maaltid", "maaltid start"]
stop_eating = ["faerdig", "stop maaltid"]
start_activity = ["start step", "start exercise"]
stop_activity = ["stop step"]
end = ["ud", "exit"]
start = ["ind i kammer"]

keywords_dict = {
    'sleeping': (["seng", "sleeping", "bed", "sove"], 1),
    'eating': (["start maaltid", "maaltid start"], 2),
    'stop_eating': (["faerdig", "stop maaltid"], 0),
    'activity': (["start step", "start exercise"], 3),
    'stop_activity': (["stop step"], 0),
    'ree_start': (["start REE", "Start REE"], 4),
    'ree_stop': (["stop REE"], 0),
    'end': (["ud", "exit"], 0),
    'start': (["ind i kammer"], 0)
}

dict_protocol = {}
for index, row in df_note.iterrows():
    for category, (keywords, value) in keywords_dict.items():
        if any(word in row['Comment'] for word in keywords):
            dict_protocol[row["datetime"]] = value
            #print("change to", value, row['datetime'])
            
protocol_list = sorted((ts, value) for ts, value in dict_protocol.items())
print(protocol_list)

current_protocol = 0    
current_index = 0  

for index, row in df_room1.iterrows():
    # While there are more timestamps and the current row's datetime is greater than or equal to the timestamp
    while (current_index < len(protocol_list) and 
           row['datetime'] >= protocol_list[current_index][0]):
        current_protocol = protocol_list[current_index][1]  # Update current protocol
        current_index += 1  # Move to the next timestamp

    df_room1.at[index, 'protocol'] = current_protocol
            

In [ ]:
display(df_room1)
df_room1.to_csv("./tmp/df_room1.csv")